In [1]:
#Load API keys for OpenAI and HuggingFaceHub
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
#Load the documents we want to prompt an LLM about
from langchain.document_loaders import TextLoader
loader = TextLoader('./vector_db_documentation.txt')
#loader = TextLoader('./State_of_the_union.txt')
doc = loader.load()

In [10]:
#Chunk the documents into 500 character chunks using langchain's text splitter "RucursiveCharacterTextSplitter"
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

chunks = text_splitter.split_documents(doc)
print(chunks[0]) #split_documents produces a list of all the chunks created, printing out first chunk for example


page_content="Algorithms Several algorithms can facilitate the creation of a vector index. Their common goal is to enable fast querying by creating a data structure that can be traversed quickly. They will commonly transform the representation of the original vector into a compressed form to optimize the query process.  However, as a user of Pinecone, you don't need to worry about the intricacies and selection of these various algorithms. Pinecone is designed to handle all the complexities and algorithmic" metadata={'source': './vector_db_documentation.txt'}


In [11]:
#The chunks must now be embedded and stored in a vector store
#Here we use HuggingFaceEmbeddings to embed the chunks, and FAISS (Facebook AI Similarity Search) as a vector store where the embedded vectors are stored
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings()
vec_db = FAISS.from_documents(chunks, embeddings) # vec_db is the vector store where the embeddings are stored


In [12]:
#An example of using similarity search directly on FAISS vector store
#The search uses Euclidean similarity search, which measures distance between two points in vector space. Measures how similar vectors are
query = "what are the strengths of a vector database"
#query = "what did president obama say about the nations strengths?"

query_sim = vec_db.similarity_search(query) #query_sim holds results of the similarity search, the closest related chunks to the query.
print(query_sim)

[Document(page_content='the emergence of vector databases as the computation engine that allows us to interact effectively with vector embeddings in our applications.  Vector databases are purpose-built databases that are specialized to tackle the problems that arise when managing vector embeddings in production scenarios. For that reason, they offer significant advantages over traditional scalar-based databases and standalone vector indexes.  In this post, we reviewed the key aspects of a vector database, including', metadata={'source': './vector_db_documentation.txt'}), Document(page_content="speed up the filtering tasks. Balancing the trade-offs between search performance and filtering accuracy is essential for providing efficient and relevant query results in vector databases.  Database Operations Unlike vector indexes, vector databases are equipped with a set of capabilities that makes them better qualified to be used in high scale production settings. Let's take a look at an over

In [13]:
#get requirements for question answer chain with LLM
from langchain import HuggingFaceHub
from langchain.llms import OpenAI

#instantiate two llm models (OpenAI text-davinci-003, HuggingFaceHub google/flan-t5-xxl(designed for short answers)) 
llm_openai = OpenAI(model="text-davinci-003", max_tokens=512)
llm_flan = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

In [7]:
print(llm_openai)
print(llm_flan)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
HuggingFaceHub
Params: {'repo_id': 'google/flan-t5-xxl', 'task': None, 'model_kwargs': {'temperature': 0.5, 'max_length': 512}}


In [14]:
from langchain.chains.question_answering import load_qa_chain

#create the chain for each model using langchain load_qa_chain
chain_openAI = load_qa_chain(llm_openai, chain_type="stuff")
chain_HuggingFaceHub = load_qa_chain(llm_flan, chain_type="stuff")

#example prompts
#query = "what year of presidency is this for Obama?"
#query = "what did president obama say about the nations strengths?"
#query = "what are the four big questions the country needs to answer?"

#query = "what is a vector database?"
query = "what are the strengths of a vector database?"
#query = "when should I use a vector store?"

query_sim = vec_db.similarity_search(query) #Gather the most related chunks to the query

#run the chain on the query and the related chunks from the documentation
print("OpenAI Response: ")
print(chain_openAI.run(input_documents=query_sim, question=query),'\n')
print("HuggingFaceHub Response: ")
print(chain_HuggingFaceHub.run(input_documents=query_sim, question=query))

OpenAI Response: 
 Vector databases are specialized to tackle the problems that arise when managing vector embeddings in production scenarios, and offer significant advantages over traditional scalar-based databases and standalone vector indexes. They are equipped with a set of capabilities that makes them better qualified to be used in high scale production settings, such as efficient search performance, relevance of query results, and user-friendly API and SDKs. 

HuggingFaceHub Response: 
they offer significant advantages over traditional scalar-based databases and standalone vector indexes
